In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.use('Cairo')  # for saving SVGs that Affinity Designer can parse
import matplotlib.pyplot as plt
import seaborn as sns
import dill

from candas.utils import setup_paths
from candas.style import breve
from candas.learn import ParameterSet
import gumbi as gmb

base_pth, code_pth, data_pth, rslt_pth, fig_pth = setup_paths(make_missing=False)
plt.style.use(str(breve))

%config InlineBackend.figure_format = 'retina'

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
ps = ParameterSet.load(data_pth / 'ADVI_ParameterSets_220528.pkl')
def make_pair(row):
    return '-'.join(sorted([row.FPrimer, row.RPrimer]))

data = (ps.wide
        .query('Metric == "mean"')
        .astype({'BP': float})
        .assign(PrimerPair = lambda df: df.apply(make_pair, axis=1))
        .groupby(['Target', 'PrimerPair','Reporter'])
        .mean()
        .reset_index()
        )

ds = gmb.DataSet(
    data = data,
    outputs = ['F0_lg', 'r', 'K', 'm'],
    log_vars = ['BP', 'K', 'm', 'r'],
    logit_vars = ['GC'],
    )

selected = (data
 .groupby(['PrimerPair', 'Reporter'])
 .size()
 .reset_index()
 .rename(columns={0:'Observations'})
 .sort_values('Observations', ascending=False)
 .reset_index(drop=True)
).iloc[[0, 1, 4, 5, 6, 8, 38, 39, 42]]

In [3]:
with open(rslt_pth / 'Model_LMC_predictions.pkl', 'rb') as f:
    predictions_dict = dill.load(f)
    
all_r = [predictions_dict[f'r{i}'] for i in range(9)]
BP = predictions_dict['BP']
GC = predictions_dict['GC']

limits = BP.parray(GC=[0.2, 0.8], BP=[10, 600])

In [4]:
width = 3.45
height = 3.31
figsize=(width, height)
spotsize=1
linewidth=2
ticklabelsize=8
labelsize=10
titlesize=labelsize+2

# Set rcParams for plotting
mpl.rc('xtick', labelsize=ticklabelsize)
mpl.rc('ytick', labelsize=ticklabelsize)
mpl.rc('axes', labelsize=labelsize, titlesize=titlesize, linewidth=1)

fig, axs = plt.subplots(3, 3, figsize=figsize, sharey=True, sharex=True)

reporter = 'HEX'

rnorm = mpl.colors.Normalize(vmin=0.23, vmax=1.03)

for i, (r, ax, row) in enumerate(zip(all_r, axs.flat, selected.itertuples())):
    
    plt.sca(ax)
    pp = gmb.ParrayPlotter(x=GC, y=BP, z=r, 
                        #    x_scale='standardized',
                           y_scale='standardized'
                           )

    step = 0.05
    levels=np.arange(np.floor(rnorm.vmin/step), np.ceil(rnorm.vmax/step)+1)*step
    cs = pp(plt.contourf, levels=levels, cmap='flare_r', norm=rnorm)
    # pp.colorbar(cs)

    yticks = BP.parray(BP=[10, 30, 100, 300])
    ax.set_yticks(yticks['BP'].z.values())
    ax.set_yticklabels(map(int, yticks.values()))
    
    if i%3 != 0:
        ax.set_ylabel('')
    if i//3 != 2:
        ax.set_xlabel('')

    gc = (ds.wide
          .query('Reporter == @row.Reporter and PrimerPair == @row.PrimerPair')
          .GC
         )
    bp = (ds.wide.z
          .query('Reporter == @row.Reporter and PrimerPair == @row.PrimerPair')
          .BP
         )

    ax.scatter(gc, bp, c='0.5', cmap='flare_r', norm=rnorm, s=spotsize)

    ax.set_xlim([0.2, 0.8])

    cs = ax.contour(GC.values(), BP.z.values(), r.σ, levels=[0.05, 0.10, 0.15, 0.20, 0.25],
                    colors='0.2', linestyles='--', linewidths=1)
    ax.clabel(cs, fontsize=ticklabelsize);
    

mar_l=0.56
mar_r=0.17
mar_t=0.22
mar_b=0.48

plt.subplots_adjust(
    left=mar_l / width,
    right=1 - mar_r / width,
    top=1 - mar_t / height,
    bottom=mar_b / height,
)

plt.savefig(fig_pth / 'Fig2d LMC r model.png', dpi=300, transparent=True);
plt.savefig(fig_pth / 'Fig2d LMC r model.svg', dpi=300, transparent=True);
    